In [1]:
pip install selenium

     |████████████████████████████████| 904 kB 3.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [128]:
import selenium
from selenium.webdriver.common.keys import Keys
import os
import time

In [2]:
PATH= '../chromedriver'

In [54]:
os.getcwd()

'/Users/angelicadegaetano/Documents/Flatiron/Capstone/Hike scrape try'

In [129]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import csv

In [31]:
# genres = ['fiction', 'action', 'documentaries']
# book_table = []
# review_table = []
# book_urls = []

In [150]:
def close_modal_by_click(driver):
    body_element =driver.find_element_by_tag_name("body")
    action = webdriver.common.action_chains.ActionChains(driver)
    action.move_to_element_with_offset(body_element, 0, 0)
    action.click()
    action.perform()

In [151]:
def get_book_urls_for_genre(driver, genre):
    book_urls = []
    driver.get('https://www.goodreads.com/genres/{}'.format(genre))
    element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "coverWrapper"))
    )
    elements = driver.find_elements_by_class_name('coverWrapper')
    for element in elements:
        a_tag = element.find_element_by_tag_name("a")
        href_url = a_tag.get_attribute('href')
        book_urls.append(href_url)
    return book_urls

In [139]:
def get_new_book_table_entry(driver):
    try:
        title = driver.find_element_by_id("bookTitle").text
        author = driver.find_element_by_id("bookAuthors").text
        average_rating = driver.find_element_by_xpath('//span[@itemprop="ratingValue"]').text
        book_description_container = driver.find_element_by_id("description")
        book_description = ''
        try:
            book_description_container.find_element_by_css_selector('a').click()
            book_description_container.find_element_by_xpath("//a[contains(text(),'...more')]").click()
            book_description = driver.find_element_by_id("description").find_elements_by_css_selector('span')[1].text
        except  Exception as e:
            print(e)
            print('more description click')
            book_description = driver.find_element_by_id("description").find_elements_by_css_selector('span')[0].text
        pages = driver.find_element_by_xpath('//span[@itemprop="numberOfPages"]').text
        path = driver.current_url
        book_info = {
            "genre": current_genre,
            "title": title,
            "author": author,
            "pages": pages,
            "unique_key": path,
            "average_rating": average_rating,
            "book_description": book_description
        }
    except Exception as e:
        print(e)
        print('except table')
        return None
    return book_info
    

In [157]:
def get_new_review_table_entries(driver):
    new_entries = []
    next_button = True
    max_review_pages = 7
    current_review_page = 1
    while(next != None and max_review_pages >= current_review_page):
        try:
            next_button = driver.find_element_by_class_name("next_page")
        except:
            next_button = None
        try:
            reviews_container_element = driver.find_element_by_id("bookReviews")
            path = driver.current_url
            users_review = reviews_container_element.find_elements_by_class_name("friendReviews")
            for user_review in users_review:
                review_text_container = user_review.find_element_by_class_name("readable")
                review_text = review_text_container.find_elements_by_css_selector('span')[0].text
                user_element = user_review.find_element_by_class_name("user")
                name = user_element.get_attribute("name")
                user_url = user_element.get_attribute("href")
                number_of_stars = len(user_review.find_elements_by_class_name("p10"))
                new_entries.append({
                    "user_name": name,
                    "user_unique_key": user_url,
                    "number_of_stars": number_of_stars,
                    "book_unique_key": path,
                    "review_text": review_text
                })
            time.sleep(1)
            current_review_page = current_review_page + 1
            print("current_review_page: {}, book_key: {}".format(current_review_page, path))
        except Exception as e:
            print(e)
            print('except reviews')
            return new_entries
        try:
            driver.find_element_by_class_name("next_page").click()
            WebDriverWait(driver, 3).until(
            EC.text_to_be_present_in_element(
                (By.XPATH, "//em[@class='current']"), "{}".format(current_review_page)
            ))
        except:
            next_button = None
    return new_entries

In [153]:
def close_current_book_tab(driver):
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [154]:
def save_book_table_genre(genre):
    with open('{}_book_table.csv'.format(genre), 'a+',) as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['genre', 'title', 'author', 'pages', 'unique_key','average_rating','book_description'])
        for book in book_table:
            writer.writerow([
                book["genre"], 
                book["title"], 
                book["author"], 
                book["pages"],
                book["unique_key"],
                book["average_rating"],
                book["book_description"]
            ])

In [155]:
def save_review_table_genre(genre):
    with open('{}_review_table.csv'.format(genre), 'a+',) as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['user_name', 'user_unique_key', 'number_of_stars', 'book_unique_key', 'review_text'])
        for review in review_table:
            writer.writerow([
                review["user_name"], 
                review["user_unique_key"], 
                review["number_of_stars"], 
                review["book_unique_key"],
                review["review_text"]
            ])

In [156]:
genres = ['fiction']
# done_genres = ['fiction', 'non-fiction','romantic', 'history-and-politics', 'biography-memoir', 'science-fiction-fantasy', 'novella', 'poetry', 'anthologies', 
#'historical-fiction', 'contemporary']
driver = webdriver.Chrome(PATH)
has_closed_modal = False
try:
    for current_genre in genres:
        book_table = []
        review_table = []
        book_urls = []
        b_urls = get_book_urls_for_genre(driver, current_genre)
        for b_url in b_urls:
            driver.execute_script("window.open('{}', 'new_window')".format(b_url))
            driver.switch_to.window(driver.window_handles[1])
            if not has_closed_modal:
                time.sleep(1)
                close_modal_by_click(driver)
                has_closed_modal = True
            new_entry = get_new_book_table_entry(driver)
            if new_entry == None:
                close_current_book_tab(driver)
                continue
            book_table.append(new_entry)
            review_table = review_table + get_new_review_table_entries(driver)
            close_current_book_tab(driver)
        save_book_table_genre(current_genre)
        save_review_table_genre(current_genre)
except Exception as e:
    print(e)
    print('except')
print('Success!')

current_review_page: 7, book_key: https://www.goodreads.com/book/show/50358031-leave-the-world-behind
current_review_page: 8, book_key: https://www.goodreads.com/book/show/50358031-leave-the-world-behind
current_review_page: 9, book_key: https://www.goodreads.com/book/show/50358031-leave-the-world-behind
current_review_page: 10, book_key: https://www.goodreads.com/book/show/50358031-leave-the-world-behind
current_review_page: 11, book_key: https://www.goodreads.com/book/show/50358031-leave-the-world-behind
current_review_page: 12, book_key: https://www.goodreads.com/book/show/50358031-leave-the-world-behind
current_review_page: 7, book_key: https://www.goodreads.com/book/show/49504061-the-once-and-future-witches
current_review_page: 8, book_key: https://www.goodreads.com/book/show/49504061-the-once-and-future-witches
current_review_page: 9, book_key: https://www.goodreads.com/book/show/49504061-the-once-and-future-witches
current_review_page: 10, book_key: https://www.goodreads.com/boo

KeyboardInterrupt: 

In [98]:
# import csv
# with open('fiction_review_table.csv', 'w',) as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(['user_name', 'user_unique_key', 'number_of_stars', 'book_unique_key'])
#     for review in review_table:
#         writer.writerow([
#             review["user_name"], 
#             review["user_unique_key"], 
#             review["number_of_stars"], 
#             review["book_unique_key"]
#         ])


In [56]:
# with open('fiction_book_table.csv', 'w',) as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(['genre', 'title', 'author', 'pages', 'unique_key'])
#     for book in book_table:
#         writer.writerow([
#             book["genre"], 
#             book["title"], 
#             book["author"], 
#             book["pages"],
#             book["unique_key"]
#         ])